In [238]:
import numpy as np
import random
from sklearn.datasets.samples_generator import make_classification
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
from collections import Counter
from pprint import pprint
from random import randrange
from sklearn.tree import DecisionTreeClassifier

In [283]:
dataset = pd.read_csv('zoo.csv',
                      names=['animal_name','hair','feathers','eggs','milk',
                                                   'airbone','aquatic','predator','toothed','backbone',
                                                  'breathes','venomous','fins','legs','tail','domestic','catsize','class',])
dataset=dataset.drop('animal_name',axis=1)

In [318]:
dataset.head()

,hair,feathers,eggs,milk,airbone,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [248]:
def entropy(target_col):
    """
    Считаем энтропийный критерий.
    Параметром функции является target_col - целевой столбец по которому нужно посчитать критерий
    """
    #узнаем уникальные элементы и их количество в целевом столбце
    elements,counts = np.unique(target_col,return_counts = True)
    #считаем критерий энтропии в цикле
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [249]:
def InfoGain(data,split_attribute_name,target_name='class'):
    """
    Считаем функционал качества, 3 параметра:
    1. data - входной датасет
    2. split_attribute_name - объект для которого считаем IG
    3. target_name - целевая функция. по дефалту ставим class (под датасет)
    """    
    #Считаем критерии для всего дата сета (тут можно сделать "переключалку" в зависимости от критерия)
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Считаем значения для сплита 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Считаем взвешеный критерий
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Считаем IG
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [250]:
def ID3(data,originaldata,features,target_attribute_name='class',parent_node_class = None):
    """
    1. data - исходный датасет. При первом запуске равен исходному датасету.
    2. originaldata - исходный датасет. Нужен что бы если первый пункт пернул пусто
    3. features - пространство признаков датасета. Нужно для рекурсивного вызова функции - разделение по веткам
    4. target_attribute_name - целевой атрибут
    5. parent_node_class - значение целевого атрибута ветки. нужно для рекурсивного вызова - когда ветка кончилась и мы хотим вернуть лист
    """   
    #Определяем критерий остановки -> Если что-то из этого выполнено, то возвращаем лист
    
    #Если все target_values одинаковые, то возвращаем это значение
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #Если датасет пустой, то значение целевого атрибута из исходного датасета
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    # Если feature пустое, то вернуть значение из родительсткого узла - parent_node_class
    
    elif len(features) ==0:
        return parent_node_class
    
    #Если ничего из вышеперечисленного не выполняется, то строим дерево.
    
    else:
        #Ставим значение по умолчанию для этого узла 
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Идем лушее features для разделения датасета
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features]#Знаение IG для объектов датасета
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Создает дерево. Корень best_feature с макс IG при первом вызове
        tree = {best_feature:{}}
        
        
        #Убираем лучшее best_feature по IG из features
        features = [i for i in features if i != best_feature]
        
        #Спускаемся от корневой ветки по всем возможным features
        
        for value in np.unique(data[best_feature]):
            value = value
            #Разделим датасет по features с наибольшим IG = создаем sub_data
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Вызываем ID3 для каждого sub_data - пошла рекурсия
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            
            #Добавляем ветки из sub_data под корень
            tree[best_feature][value] = subtree
            
        return(tree)   

In [251]:
def predict(query,tree,default = 1):
    """
    Прогноз
    Рекурсивный спуск по дереву в поисках ответа - ветка или лист
    """
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

In [252]:
def train_test_split(dataset):
    """
    Разделяем датасет на тренировочные и тестовые данные
    """
    training_data = dataset.iloc[:80].reset_index(drop=True)
    testing_data = dataset.iloc[80:].reset_index(drop=True)
    return training_data,testing_data

In [253]:
def test(data,tree):
    #СОздаем запрос, удалив целевую функцию и переведем в словарик
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Пустой датафрейм с предсказанием
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Считаем accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data['class'])/len(data))*100,'%')

In [254]:
training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 

In [255]:
tree = ID3(training_data,training_data,training_data.columns[:-1])
pprint(tree)
test(testing_data,tree)

{'legs': {0: {'fins': {0.0: {'toothed': {0.0: 7.0, 1.0: 3.0}},
                       1.0: {'eggs': {0.0: 1.0, 1.0: 4.0}}}},
          2: {'hair': {0.0: 2.0, 1.0: 1.0}},
          4: {'hair': {0.0: {'toothed': {0.0: 7.0, 1.0: 5.0}}, 1.0: 1.0}},
          6: {'aquatic': {0.0: 6.0, 1.0: 7.0}},
          8: 7.0}}
The prediction accuracy is:  85.71428571428571 %


In [256]:
train_features = df.iloc[:80,:-1]
test_features = df.iloc[80:,:-1]
train_targets = df.iloc[:80,-1]
test_targets = df.iloc[80:,-1]

In [257]:
tree = DecisionTreeClassifier(criterion = 'entropy').fit(train_features,train_targets)

In [258]:
prediction = tree.predict(test_features)

In [259]:
print("The prediction accuracy is: ",tree.score(test_features,test_targets)*100,"%")

The prediction accuracy is:  80.0 %


Другой вариант решения - можно не смотреть

In [366]:
dataset_arr = dataset.to_numpy()

In [290]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [291]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [292]:
split = get_split(dataset_arr)#data_
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X4 < 1.000]


In [293]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [294]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [295]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [296]:
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [377]:
tree = build_tree(dataset_arr, 5, 10)#data_
print_tree(tree)
#print(tree)

[X4 < 1.000]
 [X2 < 1.000]
  [X12 < 1.000]
   [X9 < 1.000]
    [X5 < 1.000]
     [7]
     [6]
    [3]
   [X1 < 0.000]
    [4]
    [4]
  [X1 < 0.000]
   [2]
   [2]
 [X1 < 1.000]
  [1]
  [X1 < 1.000]
   [1]
   [1]


In [298]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [363]:
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
return correct / float(len(actual)) * 100.0

In [392]:
max_depth = 5
min_size = 10
ttt = decision_tree(dataset_arr[:80], dataset_arr[80:,:-1], max_depth, min_size)

In [393]:
#dataset_arr[80:,-1:]

In [395]:
accuracy_metric(dataset_arr[80:,-1:], ttt)

80.95238095238095

Тест второго алгоритма с генерированным датосетом

In [398]:
x, y = make_classification(n_samples=100, random_state=20, n_informative=2, n_features=2, n_redundant = 0
                          )
m, n = np.shape(x)

In [399]:
x = np.c_[ np.ones(m), x]

In [400]:
data_ = np.c_[x,y]
df = pd.DataFrame(data=data_)
df.columns = ['a','x1','x2','y']
df.head()

,a,x1,x2,y
0,1.0,-1.582773,0.684918,0.0
1,1.0,-2.052907,0.401974,0.0
2,1.0,1.268808,-1.330203,1.0
3,1.0,0.666392,2.863175,0.0
4,1.0,0.907552,0.384912,1.0


In [401]:
max_depth = 5
min_size = 10
ttt = decision_tree(data_[:80], data_[80:,:-1], max_depth, min_size)
accuracy_metric(data_[80:,-1:], ttt)

80.0

In [402]:
tree = build_tree(data_, max_depth, min_size)
print_tree(tree)

[X2 < 0.418]
 [X2 < -0.123]
  [X3 < 1.443]
   [X1 < 1.000]
    [0.0]
    [0.0]
   [0.0]
  [0.0]
 [X3 < 2.046]
  [X3 < -3.587]
   [0.0]
   [X2 < 0.535]
    [1.0]
    [X2 < 1.193]
     [1.0]
     [1.0]
  [0.0]


In [ ]:
#Вот это я уже не смог разобрать

In [396]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [403]:
# evaluate algorithm
n_folds = 17
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset_arr, decision_tree, n_folds, max_depth, min_size)#data_
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Доп материалы
# https://www.python-course.eu/Decision_Trees.php
# https://www.machinelearningmastery.ru/implement-decision-tree-algorithm-scratch-python/
